In [1]:
# mpu6050_gyro.py
import tdwf
import time
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# --- Registri principali (interi) ---
PWR_MGMT1     = 0x6B
PWR_MNGMT2    = 0x6C
WHO_AM_I      = 0x75
SMPLRT_DIV    = 0x19
CONFIG        = 0x1A
GYRO_CONFIG   = 0x1B
GYRO_XOUT_H   = 0x43  # burst read 6 bytes: XH XL YH YL ZH ZL
SAD           = 0x68

# --- Parametri giroscopio (FS_SEL e sensibilità) ---
# GYRO_CONFIG bits (FS_SEL):
# 0x00 -> ±250 °/s  -> 131.0 LSB/(°/s)
# 0x08 -> ±500 °/s  -> 65.5  LSB/(°/s)
# 0x10 -> ±1000 °/s -> 32.8  LSB/(°/s)
# 0x18 -> ±2000 °/s -> 16.4  LSB/(°/s)
# Qui uso ±250 °/s per miglior risoluzione (modifica se ti serve altro)
GYRO_CONFIG_VALUE = 0x00
GYRO_SENSITIVITY = 131.0  # LSB per °/s per register map for FS_SEL=0

def twos_comp(val, bits=16):
    """Two's complement conversion for `bits`-bit signed integer."""
    if val & (1 << (bits - 1)):
        val -= 1 << bits
    return val

def read_gyro(sht):
    """Burst-read 6 bytes from GYRO_XOUT_H and return (gx, gy, gz) in °/s."""
    sht.writeread([GYRO_XOUT_H], 6)     # read XH XL YH YL ZH ZL
    hx, lx, hy, ly, hz, lz = sht.vals[0:6]
    raw_x = (hx << 8) | lx
    raw_y = (hy << 8) | ly
    raw_z = (hz << 8) | lz
    raw_x = twos_comp(raw_x, 16)
    raw_y = twos_comp(raw_y, 16)
    raw_z = twos_comp(raw_z, 16)
    gx = raw_x / GYRO_SENSITIVITY
    gy = raw_y / GYRO_SENSITIVITY
    gz = raw_z / GYRO_SENSITIVITY
    return gx, gy, gz, raw_x, raw_y, raw_z

def main():
    ad2 = tdwf.AD2()
    try:
        ad2.vdd = 3.3
        ad2.power(True)
        i2c = tdwf.I2Cbus(ad2.hdwf)
        devs = i2c.scan()
        print("I2C devices:", [hex(d) for d in devs])

        sht = tdwf.I2Cdevice(ad2.hdwf, SAD)

        # Reset / Wake / Configurazione base
        sht.write([PWR_MGMT1, 0x80])   # reset device
        time.sleep(0.5)
        sht.write([PWR_MGMT1, 0x00])   # wake up, set clock source
        time.sleep(0.05)
        sht.write([PWR_MNGMT2, 0x00])  # enable accel & gyro (temp indipendente)
        time.sleep(0.01)
        sht.write([CONFIG, 0x03])      # DLPF (opzionale)
        time.sleep(0.01)
        sht.write([SMPLRT_DIV, 0x04])  # sample rate divider (opzionale)
        time.sleep(0.01)

        # Imposta full scale del giroscopio
        sht.write([GYRO_CONFIG, GYRO_CONFIG_VALUE])
        time.sleep(0.01)

        # Verifica WHO_AM_I
        sht.writeread([WHO_AM_I], 1)
        if sht.vals[0] != 0x68:
            print("WHO_AM_I mismatch: 0x%02x" % sht.vals[0])
            return

        # Setup plot (solo velocità angolare per i tre assi)
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 8), sharex=True)
        plt.subplots_adjust(hspace=0.4)
        start = time.time()
        times, GX, GY, GZ = [], [], [], []
        ax1.set_ylabel("Gx [°/s]"); ax1.grid(True)
        ax2.set_ylabel("Gy [°/s]"); ax2.grid(True)
        ax3.set_ylabel("Gz [°/s]"); ax3.set_xlabel("Time [s]"); ax3.grid(True)
        line_gx, = ax1.plot([], [], marker='.', linestyle='-')
        line_gy, = ax2.plot([], [], marker='.', linestyle='-')
        line_gz, = ax3.plot([], [], marker='.', linestyle='-')

        running = True
        def on_key(event):
            nonlocal running
            if event.key == 'escape':
                running = False
        fig.canvas.mpl_connect("key_press_event", on_key)
        plt.show(block=False)

        print("Plotting gyroscope (°/s). Premere ESC nella finestra per fermare.")

        while running:
            gx, gy, gz, rx, ry, rz = read_gyro(sht)
            t = time.time() - start
            times.append(t)
            GX.append(gx); GY.append(gy); GZ.append(gz)

            # stampa rapida su console
            print(f"{t:6.2f}s  Gx={gx:7.2f}  Gy={gy:7.2f}  Gz={gz:7.2f}  (raw: {rx}, {ry}, {rz})")

            # aggiorna grafico
            line_gx.set_data(times, GX)
            line_gy.set_data(times, GY)
            line_gz.set_data(times, GZ)

            for ax in (ax1, ax2, ax3):
                ax.relim()
                ax.autoscale_view()

            fig.canvas.draw()
            fig.canvas.flush_events()

            time.sleep(0.02)  # loop ~50 Hz (dipende anche da SMPLRT_DIV / DLPF)

    finally:
        print("Cleaning up hardware...")
        try:
            plt.close('all')
        except:
            pass
        ad2.power(False)
        ad2.close()
        print("AD2 Power Off. Process safely terminated.")

if __name__ == "__main__":
    main()


Digilent WaveForms SDK versione 3.22.2
Dispositivo #1 [SN:210321B5D136, hdwf=1] connesso!
Configurazione #1
Bus I2C pronto...
I2C devices: ['0x68']
Plotting gyroscope (°/s). Premere ESC nella finestra per fermare.
  0.02s  Gx=   1.63  Gy=  -0.01  Gz=  -0.95  (raw: 213, -1, -125)
  0.57s  Gx=   1.57  Gy=   0.03  Gz=  -1.00  (raw: 206, 4, -131)
  0.68s  Gx=   1.69  Gy=   0.11  Gz=  -0.98  (raw: 221, 15, -128)
  0.81s  Gx=   1.63  Gy=   0.02  Gz=  -1.11  (raw: 213, 2, -146)
  0.92s  Gx=   1.67  Gy=   0.05  Gz=  -1.04  (raw: 219, 6, -136)
  1.02s  Gx=   1.63  Gy=   0.07  Gz=  -0.95  (raw: 213, 9, -124)
  1.13s  Gx=   1.63  Gy=   0.08  Gz=  -0.94  (raw: 214, 10, -123)
  1.24s  Gx=   1.66  Gy=   0.09  Gz=  -1.05  (raw: 218, 12, -138)
  1.35s  Gx=   1.66  Gy=   0.08  Gz=  -1.04  (raw: 218, 10, -136)
  1.46s  Gx=   1.68  Gy=   0.02  Gz=  -0.95  (raw: 220, 2, -125)
  1.58s  Gx=   1.65  Gy=   0.08  Gz=  -0.98  (raw: 216, 10, -128)
  1.69s  Gx=   1.63  Gy=   0.11  Gz=  -1.05  (raw: 214, 14, -137)

KeyboardInterrupt: 